## Data Acquisition and Cleaning

For background on this project, please see the [README](../README.md).

**Notebooks**
- Data Acquisition & Cleaning (this notebook)
- [Exploratory Data Analysis](./02_eda.ipynb)
- [Modeling](./03_modeling.ipynb)

**In this notebook, you'll find:**
- Dataset background info
- Data dictionary links
- Ingestion and cleaning processes

In [1]:
# Imports

import pandas as pd

In [2]:
# Reading in all files

income = pd.read_csv('../data/income_per_person.csv')
income.head()

,geo,1800,1801,1802,1803,1804,1805,1806,1807,1808,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,603,603,603,603,603,603,603,603,603,...,1530,1610,1660,1840,1810,1780,1750,1740,1800,1870
1,Albania,667,667,667,667,667,668,668,668,668,...,9530,9930,10200,10400,10500,10700,11000,11400,11900,12400
2,Algeria,715,716,717,718,719,720,721,722,723,...,12600,12900,13000,13200,13300,13500,13700,14000,13800,13700
3,Andorra,1200,1200,1200,1200,1210,1210,1210,1210,1220,...,41700,39000,42000,41900,43700,44900,46600,48200,49800,51500
4,Angola,618,620,623,626,628,631,634,637,640,...,5910,5900,5910,6000,6190,6260,6230,6030,5940,5850


In [3]:
# We need to unpivot our year columns to match the format of our other three dataframes

# Creating a list of the columns to unpivot
melt_columns = list(income.columns.drop('geo'))

# Using pd.melt to update the income df
income = pd.melt(income, id_vars='geo', var_name='year', value_vars=melt_columns, value_name='income_per_person')

# Reassigning the year column as int type
income['year'] = income['year'].astype(int)

In [4]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42267 entries, 0 to 42266
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   geo                42267 non-null  object
 1   year               42267 non-null  int64 
 2   income_per_person  42267 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 990.8+ KB


In [5]:
population = pd.read_csv('../data/population_total.csv')
population.head()

,geo,year,population
0,Afghanistan,1800,3280000
1,Afghanistan,1801,3280000
2,Afghanistan,1802,3280000
3,Afghanistan,1803,3280000
4,Afghanistan,1804,3280000


In [6]:
life_exp = pd.read_csv('../data/life_expectancy_years.csv')
life_exp.head()

,geo,year,life_expectancy
0,Afghanistan,1800,28.2
1,Afghanistan,1801,28.2
2,Afghanistan,1802,28.2
3,Afghanistan,1803,28.2
4,Afghanistan,1804,28.2


In [7]:
countries = pd.read_csv('../data/countries_edited.csv')
countries.head()

,name,id,region,sub-region
0,Afghanistan,AF,Asia,Southern Asia
1,Aland Islands,AX,Europe,Northern Europe
2,Albania,AL,Europe,Southern Europe
3,Algeria,DZ,Africa,Northern Africa
4,American Samoa,AS,Oceania,Polynesia


In [8]:
# Merging all dataframes in a nested pd.merge command
df = pd.merge(pd.merge(pd.merge(income, population, how='left', on=['geo', 'year']), life_exp, how='left', on=['geo', 'year']), countries, left_on='geo', right_on='name')

# Dropping one duplicate column
df.drop(columns = 'name', inplace=True)
df.head()

,geo,year,income_per_person,population,life_expectancy,id,region,sub-region
0,Afghanistan,1800,603,3280000,28.2,AF,Asia,Southern Asia
1,Afghanistan,1801,603,3280000,28.2,AF,Asia,Southern Asia
2,Afghanistan,1802,603,3280000,28.2,AF,Asia,Southern Asia
3,Afghanistan,1803,603,3280000,28.2,AF,Asia,Southern Asia
4,Afghanistan,1804,603,3280000,28.2,AF,Asia,Southern Asia


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39201 entries, 0 to 39200
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   geo                39201 non-null  object 
 1   year               39201 non-null  int64  
 2   income_per_person  39201 non-null  int64  
 3   population         39201 non-null  int64  
 4   life_expectancy    37590 non-null  float64
 5   id                 38982 non-null  object 
 6   region             39201 non-null  object 
 7   sub-region         39201 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 2.7+ MB


In [10]:
df[df['id'].isnull()]

,geo,year,income_per_person,population,life_expectancy,id,region,sub-region
23433,Namibia,1800,540,229000,32.4,NaN,Africa,Sub-Saharan Africa
23434,Namibia,1801,540,229000,32.4,NaN,Africa,Sub-Saharan Africa
23435,Namibia,1802,540,229000,32.4,NaN,Africa,Sub-Saharan Africa
23436,Namibia,1803,540,229000,32.4,NaN,Africa,Sub-Saharan Africa
23437,Namibia,1804,540,229000,32.4,NaN,Africa,Sub-Saharan Africa
...,...,...,...,...,...,...,...,...
23647,Namibia,2014,9630,2370000,63.7,NaN,Africa,Sub-Saharan Africa
23648,Namibia,2015,9910,2430000,64.3,NaN,Africa,Sub-Saharan Africa
23649,Namibia,2016,9810,2480000,64.8,NaN,Africa,Sub-Saharan Africa
23650,Namibia,2017,10200,2530000,65.3,NaN,Africa,Sub-Saharan Africa


In [11]:
# Looking at ID to see where the nulls fall
df[df['id'].isnull()]['geo'].value_counts()

Namibia    219
Name: geo, dtype: int64

In [12]:
df['id'].fillna('NA', inplace=True)
df.isnull().sum()

geo                     0
year                    0
income_per_person       0
population              0
life_expectancy      1611
id                      0
region                  0
sub-region              0
dtype: int64

Because all of our nulls are attributed to Namibia, we can impute with just one value here. 

*Note: The country abbreviation for Namibia is NA, which is likely why these appeared as nulls when we read in the data.*

In [13]:
# Now looking at life expectancy
df[df['life_expectancy'].isnull()]

,geo,year,income_per_person,population,life_expectancy,id,region,sub-region
657,Andorra,1800,1200,2650,NaN,AD,Europe,Southern Europe
658,Andorra,1801,1200,2650,NaN,AD,Europe,Southern Europe
659,Andorra,1802,1200,2650,NaN,AD,Europe,Southern Europe
660,Andorra,1803,1200,2650,NaN,AD,Europe,Southern Europe
661,Andorra,1804,1210,2650,NaN,AD,Europe,Southern Europe
...,...,...,...,...,...,...,...,...
36568,Tuvalu,2014,3270,10900,NaN,TV,Oceania,Polynesia
36569,Tuvalu,2015,3320,11000,NaN,TV,Oceania,Polynesia
36570,Tuvalu,2016,3390,11100,NaN,TV,Oceania,Polynesia
36571,Tuvalu,2017,3440,11200,NaN,TV,Oceania,Polynesia


In [14]:
df[df['life_expectancy'].isnull()]['geo'].value_counts()

Monaco              219
Nauru               219
Palau               219
San Marino          219
Tuvalu              219
Andorra             172
Dominica            172
Marshall Islands    172
Name: geo, dtype: int64

Based on the consistency of the values above across multiple countries, we can reasonably assume that these nulls are a result of a lack of data for some or all of the years that we're looking at. 

Unfortunately, we'll be using the life expectancy feature as a main component of our analysis, so we'll need to drop these countries that have all or a large majority of these values missing.

In [15]:
null_lifeexp_countries = list(df[df['life_expectancy'].isnull()]['geo'].unique())
df = df[df['geo'].isin(null_lifeexp_countries) == False]

df.reset_index(drop=True, inplace = True)
df.isnull().sum()

geo                  0
year                 0
income_per_person    0
population           0
life_expectancy      0
id                   0
region               0
sub-region           0
dtype: int64

Now that we've merged our dataframes and removed all nulls, we can proceed with EDA and modeling!

In [16]:
df.to_csv('../data/aggregated_df_cleaned.csv', index=False)